Some parts of `pygsti` are works-in-progress. Here, we investigate how to do the task of "model selection" within GST, essentially answering the question "Can we do a better job of modeling the experiment by changing the assumptions within GST?".

## Testing variable-gateset-dimension GST with model selection

###     Setup

In [1]:
import pygsti

In [2]:
#Load gateset and some string lists
gs_target = pygsti.io.load_gateset("tutorial_files/Example_Gateset.txt")
fiducialList = pygsti.io.load_gatestring_list("tutorial_files/Example_FiducialList.txt")
germList = pygsti.io.load_gatestring_list("tutorial_files/Example_GermsList.txt")
specs = pygsti.construction.build_spam_specs(fiducialList)
expList = [1,2,4]


In [3]:
#Create some testing gate string lists
lgstList = pygsti.construction.list_lgst_gatestrings(specs, gs_target.gates.keys())
lsgstLists = [ lgstList[:] ]
for exp in expList:
    gsList = pygsti.construction.create_gatestring_list(
                "f0+germ*exp+f1", f0=fiducialList, f1=fiducialList,
                germ=germList, exp=exp, order=['germ','f0','f1'])
    lsgstLists.append( lsgstLists[-1] +  gsList )
    
dsList = pygsti.remove_duplicates( lsgstLists[-1] )

In [4]:
#Test on fake data by depolarizing target set, increasing its dimension,
# and adding leakage to the gates into the new dimension.

gs_dataGen4 = gs_target.depolarize(gate_noise=0.1)
gs_dataGen5 = gs_dataGen4.increase_dimension(5)
leakGate = pygsti.construction.build_gate( [2,1],[('Q0',),('L0',)] , "LX(pi/4.0,0,2)","gm") # X(pi,Q0)*LX(pi,0,2)

gs_dataGen5['Gx'] = pygsti.objects.compose( gs_dataGen5['Gx'], leakGate)
gs_dataGen5['Gy'] = pygsti.objects.compose( gs_dataGen5['Gy'], leakGate)
print gs_dataGen5.gates.keys()

#Some debugging...
#NOTE: with LX(pi,0,2) above, dim 5 test will choose a dimension 3 gateset, which may be sensible
#       looking at the gate matrices in this case... but maybe LX(pi,...) is faulty?
#print gs_dataGen4
#print gs_dataGen5

#Jmx = GST.JOps.jamiolkowski_iso(gs_dataGen4['Gx'])
#Jmx = GST.JOps.jamiolkowski_iso(gs_dataGen5['Gx'],dimOrStateSpaceDims=[2,1])
#print "J = \n",Jmx
#print "evals = ",eigvals(Jmx)

dsFake4 = pygsti.construction.generate_fake_data(gs_dataGen4, dsList, nSamples=1000000, sampleError="binomial", seed=1234)
dsFake5 = pygsti.construction.generate_fake_data(gs_dataGen5, dsList, nSamples=1000000, sampleError="binomial", seed=1234)

['Gi', 'Gx', 'Gy']


In [5]:
print "Number of gates = ",len(gs_target.gates.keys())
print "Number of fiducials =",len(fiducialList)
print "Maximum length for a gate string in ds =",max(map(len,dsList))
print "Number of LGST strings = ",len(lgstList)
print "Number of LSGST strings = ",map(len,lsgstLists)

Number of gates =  3
Number of fiducials = 6
Maximum length for a gate string in ds = 30
Number of LGST strings =  92
Number of LSGST strings =  [92, 488, 884, 1280]


### Test using dimension-4 fake data

In [6]:
#Run LGST to get an initial estimate for the gates in gs_target based on the data in ds
# NOTE: with nSamples less than 1M (100K, 10K, 1K) this routine will choose a higher-than-4 dimensional gateset
ds = dsFake4
gs_lgst4 = pygsti.do_lgst(ds, specs, targetGateset=gs_target, svdTruncateTo=4, verbosity=3)
gs_lgst6 = pygsti.do_lgst(ds, specs, targetGateset=gs_target, svdTruncateTo=6, verbosity=3)

#Print chi^2 of 4-dim and 6-dim estimates
chiSq4 = pygsti.chi2(ds, gs_lgst4, lgstList, minProbClipForWeighting=1e-4)
chiSq6 = pygsti.chi2(ds, gs_lgst6, lgstList, minProbClipForWeighting=1e-4)
print "LGST dim=4 chiSq = ",chiSq4
print "LGST dim=6 chiSq = ",chiSq6

# Least squares GST with model selection
gs_lsgst = pygsti.do_iterative_mc2gst_with_model_selection(ds, gs_lgst4, 1, lsgstLists, verbosity=2,
                                                           minProbClipForWeighting=1e-3, probClipInterval=(-1e5,1e5))

LGST: Singular values of I_tilde (truncating to first 4 of 6) = 
[  3.00484791e+00   8.25822588e-01   6.70903966e-01   6.65330643e-01
   1.32123551e-03   7.62101431e-04]

--- LGST ---
LGST: Singular values of I_tilde (truncating to first 6 of 6) = 
[  3.00484791e+00   8.25822588e-01   6.70903966e-01   6.65330643e-01
   1.32123551e-03   7.62101431e-04]
LGST: Padding target B with sqrt of low singular values of I_tilde: 
[ 0.00132124  0.0007621 ]

--- LGST ---
LGST dim=4 chiSq =  584.010103572
LGST dim=6 chiSq =  218.874929215

--- Iterative LSGST: Beginning iter 1 of 4 : 92 gate strings ---


--- Least Squares GST with model selection (starting dim = 4) ---
--- Least Squares GST ---
  Sum of Chi^2 = 58.5712 (92 data params - 40 model params = expected mean of 52; p-value = 0.247042)


Dim 4: chi^2 = 58.5712, nGateStrings=92, nParams=56 (so expected mean = 36)
--- Least Squares GST ---
  Sum of Chi^2 = 5.5151e+06 (92 data params - 24 model params = expected mean of 68; p-value = 0)


Rejected dim 3: chi^2 = 5.5151e+06 (+5.51505e+06 w.r.t. expected mean of 92 strings - 33 params = 59) (dChi^2=5515046, 2*dParams=-46)
--- Least Squares GST ---
  Sum of Chi^2 = 42.5285 (92 data params - 60 model params = expected mean of 32; p-value = 0.101024)


Rejected dim 5: chi^2 = 42.5285 (+35.5285 w.r.t. expected mean of 92 strings - 85 params = 7) (dChi^2=-16, 2*dParams=58)

--- Iterative LSGST: Beginning iter 2 of 4 : 488 gate strings ---


...


Rejected dim 5: chi^2 = 817.373 (+18.3731 w.r.t. expected mean of 884 strings - 85 params = 799) (dChi^2=-42, 2*dParams=58)

--- Iterative LSGST: Beginning iter 4 of 4 : 1280 gate strings ---


--- Least Squares GST with model selection (starting dim = 4) ---
--- Least Squares GST ---
  Sum of Chi^2 = 1262.44 (1280 data params - 40 model params = expected mean of 1240; p-value = 0.322292)


Dim 4: chi^2 = 1262.44, nGateStrings=1280, nParams=56 (so expected mean = 1224)
--- Least Squares GST ---
  Sum of Chi^2 = 6.04917e+07 (1280 data params - 24 model params = expected mean of 1256; p-value = 0)


Rejected dim 3: chi^2 = 6.04917e+07 (+6.04904e+07 w.r.t. expected mean of 1280 strings - 33 params = 1247) (dChi^2=60490400, 2*dParams=-46)
--- Least Squares GST ---
  Sum of Chi^2 = 1219.46 (1280 data params - 60 model params = expected mean of 1220; p-value = 0.498997)


Rejected dim 5: chi^2 = 1219.46 (+24.4576 w.r.t. expected mean of 1280 strings - 85 params = 1195) (dChi^2=-42, 2*dParams=58)


In [7]:
print gs_lsgst

rho0 =    0.7071   0.0242  -0.0257   0.7454


E0 =    0.6852  -0.0149   0.0109  -0.6492


Gi = 
   1.0000        0        0        0
   0.0030   0.9001  -0.0001   0.0003
  -0.0028        0   0.8998   0.0002
  -0.0036  -0.0001   0.0002   0.9000


Gx = 
   1.0000        0        0        0
   0.0027   0.8993   0.0119  -0.0190
   0.0044   0.0243  -0.0108   0.9823
  -0.0578  -0.0184  -0.8249   0.0112


Gy = 
   1.0000        0        0        0
  -0.0041  -0.0197   0.0023  -0.9987
  -0.0025  -0.0077   0.8998   0.0041
  -0.0579   0.8113   0.0281   0.0197





### Test using dimension-5 fake data

In [8]:
#Run LGST to get an initial estimate for the gates in gs_target based on the data in ds
ds = dsFake5
gs_lgst4 = pygsti.do_lgst(ds, specs, targetGateset=gs_target, svdTruncateTo=4, verbosity=3)
gs_lgst6 = pygsti.do_lgst(ds, specs, targetGateset=gs_target, svdTruncateTo=6, verbosity=3)

#Print chi^2 of 4-dim and 6-dim estimates
chiSq4 = pygsti.chi2(ds, gs_lgst4, lgstList, minProbClipForWeighting=1e-2)
chiSq6 = pygsti.chi2(ds, gs_lgst6, lgstList, minProbClipForWeighting=1e-2)
print "LGST dim=4 chiSq = ",chiSq4
print "LGST dim=6 chiSq = ",chiSq6

# Least squares GST with model selection
gs_lsgst = pygsti.do_iterative_mc2gst_with_model_selection(ds, gs_lgst4, 1, lsgstLists, verbosity=2, minProbClipForWeighting=1e-3, probClipInterval=(-1e5,1e5), useFreqWeightedChiSq=False, regularizeFactor=1.0, check=False, check_jacobian=False)

LGST: Singular values of I_tilde (truncating to first 4 of 6) = 
[  2.36368304e+00   6.59319496e-01   4.68883491e-01   4.54819490e-01
   3.15418560e-03   8.91359166e-04]

--- LGST ---
LGST: Singular values of I_tilde (truncating to first 6 of 6) = 
[  2.36368304e+00   6.59319496e-01   4.68883491e-01   4.54819490e-01
   3.15418560e-03   8.91359166e-04]
LGST: Padding target B with sqrt of low singular values of I_tilde: 
[ 0.00315419  0.00089136]

--- LGST ---
LGST dim=4 chiSq =  1316593.37275
LGST dim=6 chiSq =  580943.33613

--- Iterative LSGST: Beginning iter 1 of 4 : 92 gate strings ---


--- Least Squares GST with model selection (starting dim = 4) ---
--- Least Squares GST ---
  Sum of Chi^2 = 143358 (92 data params - 40 model params = expected mean of 52; p-value = 0)


Dim 4: chi^2 = 143358, nGateStrings=92, nParams=56 (so expected mean = 36)
--- Least Squares GST ---
  Sum of Chi^2 = 3.0845e+06 (92 data params - 24 model params = expected mean of 68; p-value = 0)


Rejected dim 3: chi^2 = 3.0845e+06 (+3.08444e+06 w.r.t. expected mean of 92 strings - 33 params = 59) (dChi^2=2941144, 2*dParams=-46)
--- Least Squares GST ---
  Sum of Chi^2 = 48.7083 (92 data params - 60 model params = expected mean of 32; p-value = 0.0295689)


Selected dim 5: chi^2 = 48.7083 (+41.7083 w.r.t. expected mean of 92 strings - 85 params = 7) (dChi^2=-143309, 2*dParams=58)

--- Iterative LSGST: Beginning iter 2 of 4 : 488 gate strings ---


...


Rejected dim 8: chi^2 = 606.767 (-69.2326 w.r.t. expected mean of 884 strings - 208 params = 676) (dChi^2=-57, 2*dParams=94)

--- Iterative LSGST: Beginning iter 4 of 4 : 1280 gate strings ---


--- Least Squares GST with model selection (starting dim = 7) ---
--- Least Squares GST ---
  Sum of Chi^2 = 1069.19 (1280 data params - 112 model params = expected mean of 1168; p-value = 0.981746)


Dim 7: chi^2 = 1069.19, nGateStrings=1280, nParams=161 (so expected mean = 1119)
--- Least Squares GST ---
  Sum of Chi^2 = 1170.16 (1280 data params - 84 model params = expected mean of 1196; p-value = 0.697933)


Rejected dim 6: chi^2 = 1170.16 (+10.1634 w.r.t. expected mean of 1280 strings - 120 params = 1160) (dChi^2=100, 2*dParams=-82)
--- Least Squares GST ---
  Sum of Chi^2 = 988.099 (1280 data params - 144 model params = expected mean of 1136; p-value = 0.999391)


Rejected dim 8: chi^2 = 988.099 (-83.9009 w.r.t. expected mean of 1280 strings - 208 params = 1072) (dChi^2=-81, 2*dParams=94)


In [9]:
print gs_lsgst

rho0 =    0.7070  -0.1431  -0.0263   0.7496   0.0675  -0.1284   0.1251


E0 =    0.5879   0.0306   0.0454  -0.5123   0.0324   0.0916  -0.1316


Gi = 
   1.0000  -0.0001  -0.0002        0   0.0002  -0.0003  -0.0007
  -0.0618   1.2535   0.2658   0.2660  -0.2463   0.2606  -0.3594
  -0.0915   0.5259   1.2651   0.4113  -0.3287   0.3226  -0.7415
  -0.0771   0.3572   0.2339   1.1808  -0.2506   0.1884  -0.5575
   0.0465  -0.3242  -0.1805  -0.2588   1.1811  -0.1032   0.6315
  -0.0740   0.5422   0.3447   0.4411  -0.3069   1.1590  -0.9923
   0.1869  -0.9898  -0.8565  -0.6837   0.7848  -0.9839   1.1343


Gx = 
   1.0000        0        0        0        0        0        0
   0.0640   0.5570  -0.0161  -0.2529   0.2305  -0.5904  -0.4919
  -0.0152  -0.1097   0.0512   0.8100   0.3155   0.0082  -0.2093
  -0.1532  -0.0034  -0.5928   0.0811  -0.0711   0.3498   0.0356
  -0.2918   0.0299  -0.1255  -0.1639   0.7861  -0.1297  -0.5593
  -0.0029  -0.2121   0.2583  -0.4208   0.1347   0.0767  -0.8558
  -0.0404 